**Daphné Hbek, Elias Rhouzlane.**
Université Pierre Marie Curie, Paris, France


---
L'objectif est de représenter les mots-croisés comme un problème de satisfaction de contraintes (CSP) pour ensuite trouver un ensemble d'instanciations (variable = valeur) qui satisfasse l'ensemble des contraintes.

# 1 Modélisation par un CSP et résolution

## 1.1 Proposition 

Un problème de satisfaction de contraintes peut être représenté par un triplet (X, D, C) où :

- $X = \{v_1, v_2, \dots, v_n\}$, est un ensemble de $n$ variables
- $D = \{d_1, d_2, \dots, d_n\}$, est l'ensemble des $n$ domaines finis associés aux variables
- $C = \{c_1, c_2, \dots, c_m\}$, est un ensemble de $m$ contraintes

Dans le cas des mots-croisés, on peut définir les différents mots à trouver comme les variables du CSP et définir les contraintes sur et entre ces mots. Un mot est une variable contrainte par une certaine taille et où les lettres doivent être les même que tout autre mot de la grille qu'il intersecte. Le domaine de chaque variable est un ensemble de mot issus d'un dictionnaire de taille fixé à l'avance. Enfin, le domaine de chaque variable est réduit aux mots du dictionnaire de même taille que le mot à trouver. Chaque lettre dans les mots est l'une des 26 de l'alphabet en plus de caractères additionnels.

Ainsi nous avons défini notre CSP comme suivant:

- $X = \{mot_1, mot_2, \dots, mot_n\}$, est l'ensemble des mots à trouver dans la grille
- $D = \{d_1, d_2, \dots, d_n\}$ où chaque $d_i$ est un sous-ensemble du dictionnaire
- $C = \{c_1, c_2, \dots, c_m\}$, est un ensemble de contraintes sur la taille de chaque variable et de contraintes sur l'égalité de lettre aux intersections

Un exemple de CSP suivant notre modélisation serait le suivant:

| Variables | Domaines                | Contrainte unaire | Contrainte binaire             |
|-----------|-------------------------|-------------------|--------------------------------|
| $mot_1$     | {ABLE, ACID, ..., WORM} | mot.taille = 4        | intersect(mot_1, mot_2, (2,5)) |
| $mot_2$     | {ACT, AIR, ..., YOU}    | mot.taille = 3        | None                           |
| $mot_3$     | {ACROSS, ..., WRITING}  | mot.taille = 5        | None                           |

Afin d'appliquer la contrainte supplémentaire qu'un même mot ne peut apparaitre plus d'une fois dans la grille, il suffit d'utiliser la contrainte $\text{ALL-DIFF}$ et vérifier qu'à chaque instanciation d'une variable, la valeur de l'instanciation n'a pas déjà été utilisée.

Une autre méthode, plus efficace, serait de mettre à jour le domaine des variables de taille égale à la valeur de la variable instanciée et de la supprimer de ces domaines.

## 1.2 Implémentation

Aide à l'utilisation du programme et description des fonctions:

### Exemple d'utilisation (via console)

In [1]:
import classes as cl
import gestDict as dic
import gestIO as io

On récupère le dictionnaire, le fichier source est modifiable via une variable globale dans le module $\text{gestDict}$.

In [2]:
dic.recupDictionnaire()

Récupération du dictionnaire contenu dans C:\cygwin64\home\elias\git\RP/data/Dicos/133000-mots-us.txt


On sélectionne une grille existante au format txt.

In [3]:
grid = io.read_file("grille1.txt")[0]
print grid


Le fichier grille1.txt existe
0 0 0 1 1
0 0 0 0 1
0 0 0 0 0
1 0 0 0 0
1 1 0 0 0


Grille 5*5 avec 10 mots



Création de l'objet utile à la résolution de notre grille de mots-croisés avec en paramètre l'objet grille et le dictionnaire.

In [12]:
solver = cl.Solver(grid, dic.DICTIONNAIRE, random=False)

Résolution du mots-croisés par forward checking avec une étape préliminaire de AC3. L'objet retourné est un dictionnaire où chaque variable de mot a été instancié en respect du domaine et des contraintes.

In [14]:
solver.ac3()
instance_ = solver.forwardChecking(first=True)
print "Variable : Valeur"
for index_variable, valeur in instance_.items():
    print "{:8} : {}".format(index_variable, valeur)

Variable : Valeur
       1 : AGO
       2 : GIVE
       3 : ELOPE
       4 : TIEL
       5 : DEL
       6 : AGE
       7 : GILT
       8 : OVOID
       9 : EPEE
      10 : ELL


## 1.3 Algorithme de Forward-Checking (FC)

### Heuristique
Pour le choix de la variable à instancier nous avons choisi d'utiliser une heuristique Minimum-remaining-value (MRV) qui nous retourne la variable qui a le plus petit nombre de valeur légale et issue de son domaine.

In [6]:
help(cl.Solver.mrv)

Help on method mrv in module classes:

mrv(self, instance) unbound classes.Solver method
    Minimum-remaining-value (MRV) heuristic
    @return the variable from amongst those that have the fewest legal values



## Algorithme de Conflict BackJumping (CBJ)

# 2 Expérimentation

Nous avons appliqué notre algorithme sur les 3 grilles A, B et C, et avons enregistré les temps moyens de résolution pour AC3, FC sans AC3 préalable, FC avec AC3 préalable.


| Method    | Grille A | Grille B | Grille C |
|-----------|----------|----------|----------|
| AC3       | 0.00     | 0.00     | 0.00     |
| FC        | 0.00     | 0.00     | 0.00     |
| AC3 + CJB | 0.00     | 0.00     | 0.00     |

In [15]:
A = io.read_file("grille1.txt")[0]
B = io.read_file("grille3.txt")[0]
C = io.read_file("grille4.txt")[0]


Le fichier grille1.txt existe

Le fichier grille3.txt existe

Le fichier grille4.txt existe


Création de la fonction $\text{time_it}$ permettant d'évaluer la performance de nos algorithmes.

In [46]:
import numpy as np
import time
def time_it(functions, iterations, grid):
    time_log = np.empty((iterations,))
    for i in range(iterations):
        solver = cl.Solver(grid, dic.DICTIONNAIRE, random=True)
        start = time.time()
        for f in functions:
            f(solver)
        time_log[i] = time.time() - start
    return np.mean(time_log), np.std(time_log)

In [47]:
mean, std = time_it([lambda tmpsolver: tmpsolver.ac3()], 10, A)
print mean, std

0.184200048447 0.00960004329726


In [48]:
mean, std = time_it([lambda tmpsolver: tmpsolver.forwardChecking(first=True)], 10, A)
print mean, std

1.0746999979 0.213829400315
